# Documentation for the creation and usage of the heatpump library (hplib)




In [1]:
import hplib as hpl
import database as db

## Database Preparation
1. we downloaded all manufacturer data from https://keymark.eu/en/products/heatpumps/certified-products
2. then we unzipped the files and used the bash-skript to convert pdf into txt
3. afterwards we used this following functions to create and extent the heatpump keymark database

In [2]:
# Import keymark data and save to csv database
db.ImportKeymarkData()
# -> this creates /output/database_keymark.csv)

In [ ]:
# Reduce to climate measurement series with average climate, delete redundant entries and save to csv sub-database
db.ReduceKeymarkData('database_keymark.csv','average')
# -> this creates /output/database_keymark_average.csv)

In [ ]:
# Normalize electrical and thermal power from the keymark database to values from setpoint T_in = -7 °C and T_out = 52 °C
db.NormalizeKeymarkData('database_keymark_average.csv')
# -> this creates /output/database_keymark_average_normalized.csv)

In [ ]:
# Identify subtypes like On-Off, Inverter and 2-stages Heatpump and assign group depending on its type and subtype
db.IdentifySubtypes('database_keymark_average_normalized.csv')
# -> this creates /output/database_keymark_average_normalized_subtypes.csv)

In [9]:
#Calculate parameters p1-p3 for P_th, P_el and COP
db.CalculateFunctionParameters('database_keymark_average_normalized_subtypes.csv')
# -> this creates the final hplib-database.csv

In [10]:
#Calculate generic heatpump models for each group
db.addGeneric()
# -> this overwrites the hplib-database.csv

## Usage for simulation

In [2]:
# Define a specific heatpump model and get parameters
parameters = hpl.getParameters('Generic',1,10000)# example: hpl.getParameters('LW 101) or for generic: hpl.getParameters('Generic',1,1000)
#if generic give Group and Thermal Power at -7/52
parameters

,Model,P_th_ref [W],P_el_ref [W],Group,p1_P_th [1/°C],p2_P_th [1/°C],p3_P_th [-],p1_P_el [1/°C],p2_P_el [1/°C],p3_P_el [-],p1_COP [-],p2_COP [-],p3_COP [-]
0,Generic,10000,7837.066394,1.0,-0.030728,-0.003468,0.908301,-0.030655,0.009683,0.179736,0.149456,-0.091262,7.212956


In [3]:
# Simulate on time step
P_th, P_el, COP = hpl.simulate(-7, 52, parameters)
print('Thermal Power output: '+str(P_th)+' W')
print('Electrical Power input: '+str(P_el)+' W')
print('COP: '+str(COP))

Thermal Power output: 10000.000000000002 W
Electrical Power input: 7036.549601310043 W
COP: 1.4211510707091772


In [4]:
# Simulate a time series